# Cell Nuclei

In [1]:
import libcarna
import skimage

Get the data:

In [2]:
data = skimage.data.cells3d()
data.shape

(60, 2, 256, 256)

The data has 60 slices and 2 channels. We will visualize the nuclei channel of the data.

## Maximum Intensity Projection

In the code below, we use `normals=True` on the `volume` node; albeit this doesn't make any difference for the maximum
intensity projection (MIP), it is benefitial for other, subsequently shown rendering modes.

In [3]:
GEOMETRY_TYPE_VOLUME = 2

# Create and configure frame renderer
mip = libcarna.mip(GEOMETRY_TYPE_VOLUME, cmap='jet', sr=500)
r = libcarna.renderer(600, 450, [mip])

# Create and configure scene
root = libcarna.node()
libcarna.volume(
    GEOMETRY_TYPE_VOLUME,
    data[:, 1],
    parent=root,
    spacing=(1, 0.5, 0.5),
    local_transform=libcarna.rotate('y', 90).rotate('x', -35),
    normals=True,
)
camera = libcarna.camera(
    parent=root,
    projection=r.frustum(fov=90, z_near=1, z_far=500),
    local_transform=libcarna.translate(0, 0, 100),
)

# Render
libcarna.imshow(r.render(camera))

In [4]:
# Render as animation
libcarna.imshow(
    libcarna.animation(
        [
            libcarna.animation.swing_local(camera, amplitude=22)
        ],
        n_frames=50,
    ).render(r, camera),
)

## Direct Volume Rendering

In [5]:
dvr = libcarna.dvr(
    GEOMETRY_TYPE_VOLUME, sr=500, transl=0, diffuse=0.8,
)
dvr.cmap.clear()
dvr.cmap.write_linear_segment(
    0.1, 0.3,
    libcarna.color(0,   0, 255,   0),
    libcarna.color(0, 255, 255, 255),
)
dvr.cmap.write_linear_segment(
    0.3, 0.9,
    libcarna.color(0,   255, 255, 255),
    libcarna.color(255, 255,   0, 255),
)

libcarna.imshow(
    libcarna.animation(
        [
            libcarna.animation.swing_local(camera, amplitude=22)
        ],
        n_frames=50,
    ).render(
        libcarna.renderer(600, 450, [dvr]),
        camera,
    ),
)